In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

In [78]:
tickers = ['SPY','TLT','GLD','QQQ','EEM']
start_date = '2015-08-13'
end_date = datetime.today().strftime('%Y-%m-%d')
df = yf.download(tickers,start_date,end_date, interval='1d')['Close'].dropna()

/var/folders/y7/hf6140q96774323j7d756khw0000gn/T/ipykernel_40323/785617954.py:4: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  5 of 5 completed


In [79]:
log_returns = np.log(df/df.shift(1)).dropna()
mean_returns = log_returns.mean()*252
cov_matrix = log_returns.cov()*252
cor_matrix = log_returns.corr()
# print(log_returns,mean_returns,cov_matrix)
print(mean_returns)
print(cor_matrix)

Ticker
EEM    0.056568
GLD    0.106241
QQQ    0.173988
SPY    0.129938
TLT   -0.009015
dtype: float64
Ticker       EEM       GLD       QQQ       SPY       TLT
Ticker                                                  
EEM     1.000000  0.164969  0.726121  0.757220 -0.133834
GLD     0.164969  1.000000  0.054403  0.044493  0.298370
QQQ     0.726121  0.054403  1.000000  0.934154 -0.126857
SPY     0.757220  0.044493  0.934154  1.000000 -0.180209
TLT    -0.133834  0.298370 -0.126857 -0.180209  1.000000


In [80]:
np.random.seed()

n_portfolio =10000
n_asset =len(tickers)

results={
    'Returns':[],
    'Volatility':[],
    'Sharpe Ratio':[],
    'Weights':[]
}

for _ in range(n_portfolio):
    weights = np.random.random(n_asset)
    weights /= np.sum(weights)
    
    portfolio_return =np.dot(weights,mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix,weights)))
    sharpe_ratio = portfolio_return/portfolio_volatility
    
    results['Returns'].append(portfolio_return)
    results['Volatility'].append(portfolio_volatility)
    results['Sharpe Ratio'].append(sharpe_ratio)
    results['Weights'].append(weights)

portfolios_df =pd.DataFrame(results)
portfolios_df.head()

,Returns,Volatility,Sharpe Ratio,Weights
0,0.107439,0.115571,0.929639,"[0.011789886309497826, 0.32276806688901577, 0...."
1,0.095881,0.115635,0.829170,"[0.1305615400091396, 0.2946184524251425, 0.304..."
2,0.095369,0.122837,0.776385,"[0.17192975344484182, 0.22195049965807723, 0.1..."
3,0.057824,0.106772,0.541564,"[0.30874791747212466, 0.24518358150237565, 0.0..."
4,0.086309,0.125149,0.689653,"[0.3219179013135924, 0.2655696358152922, 0.196..."


In [81]:
fig = px.scatter(portfolios_df,x='Volatility',y='Returns',color='Sharpe Ratio',hover_data=['Weights'])
fig.update_layout(title='Raw Efficient Frontier',xaxis_title='Volatility',yaxis_title='Returns')
fig.show()

In [82]:
max_sharpe_idx = portfolios_df['Sharpe Ratio'].idxmax()

In [83]:
max_sharpe_idx

4076